Export data from SQL database to CSV, so that R can read it for complete case analysis or impute missing values.

In [42]:
import sqlite3
import pandas as pd
import datetime
from tqdm import tqdm
import sys

sys.path.append("../../..")

from utils.constants import DatabaseConfig, TableNames

In [26]:
conn = sqlite3.connect(DatabaseConfig.DB_PATH)
cursor = conn.cursor()
primary_key = 'eid'

# Generate Table

## Covariates Table

We first create a table for covariates that will be used for survival analysis.
+ We create an `age` column based on ECG_date.

In [29]:
create_table_sql = f"""
CREATE TABLE {TableNames.COVARIATES} (
    eid INTEGER PRIMARY KEY,
    age INTEGER,
    sex INTEGER,            -- (0=Female, 1=Male)
    ethnicity INTEGER,      -- (1=White, 2=Mixed, 3=Asian, 4=Black, 5=Chinese, 6=Other)
    BMI REAL,
    smoking INTEGER,        -- (0=Never, 1=Previous, 2=Current)
    diabetes INTEGER,       -- (1=Yes, 0=No)
    systolic_bp REAL,      -- Systolic Blood Pressure
    hypertension_treatment INTEGER,
    total_chol REAL,
    hdl_chol REAL,
    education INTEGER,      -- Education Attainment 
    activity INTEGER,       -- IPAQ Activity Group (0=low, 1=moderate, 2=high)
    max_workload REAL,
    max_heart_rate REAL
);
"""
cursor.execute(create_table_sql)

# * Here we don't exclude any participants so that imputation might have more information to work with.
# Here JULIANDAY() takes a date string and return a number that can be used for calculation.
insert_data_sql = f"""
INSERT INTO {TableNames.COVARIATES}
SELECT
    c.eid,
    CAST((JULIANDAY(p.ECG_date) - JULIANDAY(p.birth_date)) / 365.25 AS INTEGER) as age,  -- consider leap years
    c."31-0.0" AS sex,
    p.ethnicity,
    p.BMI,
    c."20116-0.0" AS smoking,
    c."2443-0.0" AS diabetes,
    c."4080-0.0" AS systolic_bp,
    p.hypertension_treatment,
    c."30690-0.0" AS total_chol,
    c."30760-0.0" AS hdl_chol,
    c."6138-0.0" AS education,
    c."22032-0.0" AS activity,
    e."6032-0.0" AS max_workload,
    e."6033-0.0" AS max_heart_rate
FROM
    {TableNames.CONFOUNDERS} c
    INNER JOIN {TableNames.PROCESSED} p ON c.eid = p.eid
    INNER JOIN {TableNames.ECG} e ON c.eid = e.eid;
"""
cursor.execute(insert_data_sql)
conn.commit()

## Status Table

We will also create a table to record the status of each participant. This will mainly be used for selection of eligible participants as well as survival analysis.

+ We create `ecg_hrv_ok` column that should be 1 if ECG is uncorrupted and HRV indices are available.
+ We create `ecg_before_cvd` column. It is 1 when participant has CVD and the CVD occurred before taking ECG.
+ We also creates `time` column based on ECG_date.

In [56]:
create_table_sql = f"""
CREATE TABLE {TableNames.STATUS} (
    eid INTEGER PRIMARY KEY,
    event INTEGER,
    time DATE,  -- in days
    ecg_hrv_ok INTEGER,
    ecg_before_cvd INTEGER,
    statins INTEGER
);
"""
cursor.execute(create_table_sql)

query_sql = f"""
SELECT {primary_key}, CVD, ECG_date, CVD_date, statins, Censor_date, HRV_available 
FROM {TableNames.PROCESSED};
"""
cursor.execute(query_sql)

for row in tqdm(cursor.fetchall()):
    eid = row[0]
    CVD = row[1]
    ECG_date = row[2]
    CVD_date = row[3]
    statins = row[4]
    Censor_date = row[5]
    HRV_available = row[6]

    assert (CVD == 0 and CVD_date is None) or (CVD == 1 and CVD_date is not None), "CVD_date must be consistent with CVD"
    event = 0 if CVD == 0 else 1

    if ECG_date is None:
        time = None
    else:
        ECG_date_time = datetime.datetime.strptime(ECG_date, "%Y-%m-%d")
        if CVD_date is None:
            Censor_date_time = datetime.datetime.strptime(Censor_date, "%Y-%m-%d")
            time = (Censor_date_time - ECG_date_time).days
        else:
            CVD_date_time = datetime.datetime.strptime(CVD_date, "%Y-%m-%d")
            time = (CVD_date_time - ECG_date_time).days

    ecg_hrv_ok = 1 if HRV_available == 1 and ECG_date is not None else 0

    if CVD == 1 and ECG_date is not None and ECG_date > CVD_date:
        ecg_before_cvd = 1
    else:
        ecg_before_cvd = 0

    statins = 1 if statins == 1 else 0

    insert_sql = f"""
    INSERT INTO {TableNames.STATUS}
    VALUES (?, ?, ?, ?, ?, ?);
    """
    cursor.execute(insert_sql, (eid, event, time, ecg_hrv_ok, ecg_before_cvd, statins))

conn.commit()

100%|██████████| 77888/77888 [00:00<00:00, 138781.32it/s]


# Count Missing Rate

We can also count the missing rate in the `Covariates` table before imputation.

In [40]:
# Full data
cursor.execute(f"PRAGMA table_info({TableNames.COVARIATES});")
columns = [row[1] for row in cursor.fetchall()]
columns = [col for col in columns if col != primary_key]

query_sql = f"""
SELECT {primary_key}, 
       {', '.join(f'SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_null' for col in columns)}, 
       COUNT(*) AS total
FROM {TableNames.COVARIATES}
"""

cursor.execute(query_sql)
print("Full data:")
for row in tqdm(cursor.fetchall()):
    eid = row[0]
    values = row[1:-1]
    total = row[-1]
    
    for i, value in enumerate(values):
        print(f"{columns[i]}: {value} / {total} -> {(value / total * 100):.2f}%")

Full data:


100%|██████████| 1/1 [00:00<00:00, 11650.84it/s]

age: 15961 / 77888 -> 20.49%
sex: 0 / 77888 -> 0.00%
ethnicity: 0 / 77888 -> 0.00%
BMI: 278 / 77888 -> 0.36%
smoking: 209 / 77888 -> 0.27%
diabetes: 209 / 77888 -> 0.27%
systolic_bp: 458 / 77888 -> 0.59%
hypertension_treatment: 486 / 77888 -> 0.62%
total_chol: 6238 / 77888 -> 8.01%
hdl_chol: 9657 / 77888 -> 12.40%
education: 209 / 77888 -> 0.27%
activity: 14241 / 77888 -> 18.28%
max_workload: 1 / 77888 -> 0.00%
max_heart_rate: 17 / 77888 -> 0.02%


In [58]:
# Qualified data
# * This exclusion criteria may be modified for sensitivity analysis.
exclusion_criteria = "WHERE s.statins = 0 AND s.ecg_hrv_ok = 1 AND s.ecg_before_cvd == 0"
cursor.execute(f"PRAGMA table_info({TableNames.COVARIATES});")
columns = [row[1] for row in cursor.fetchall()]
columns = [col for col in columns if col != primary_key]

query_sql = f"""
SELECT c.{primary_key}, 
       {', '.join(f'SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_null' for col in columns)}, 
       COUNT(*) AS total
FROM {TableNames.COVARIATES} c INNER JOIN {TableNames.STATUS} s ON c.eid = s.eid
{exclusion_criteria};
"""

cursor.execute(query_sql)
print("Qualified data:")
for row in tqdm(cursor.fetchall()):
    eid = row[0]
    values = row[1:-1]
    total = row[-1]
    
    for i, value in enumerate(values):
        print(f"{columns[i]}: {value} / {total} -> {(value / total * 100):.2f}%")

Qualified data:


100%|██████████| 1/1 [00:00<00:00, 9425.40it/s]

age: 0 / 35159 -> 0.00%
sex: 0 / 35159 -> 0.00%
ethnicity: 0 / 35159 -> 0.00%
BMI: 18 / 35159 -> 0.05%
smoking: 68 / 35159 -> 0.19%
diabetes: 68 / 35159 -> 0.19%
systolic_bp: 157 / 35159 -> 0.45%
hypertension_treatment: 183 / 35159 -> 0.52%
total_chol: 2661 / 35159 -> 7.57%
hdl_chol: 4262 / 35159 -> 12.12%
education: 68 / 35159 -> 0.19%
activity: 5764 / 35159 -> 16.39%
max_workload: 0 / 35159 -> 0.00%
max_heart_rate: 1 / 35159 -> 0.00%


# Export data to CSV

## Full Data

In [43]:
query_sql = f"""
SELECT * FROM {TableNames.COVARIATES};
"""
cursor.execute(query_sql)

columns = [description[0] for description in cursor.description]
data = cursor.fetchall()

df = pd.DataFrame(data, columns=columns)
df.to_csv("full_data.csv", index=False)


## Data for Survival Analysis

In [65]:
# * This exclusion criteria may be modified for sensitivity analysis.
exclusion_criteria = "WHERE s.statins = 0 AND s.ecg_hrv_ok = 1 AND s.ecg_before_cvd == 0"

cursor.execute(f"PRAGMA table_info({TableNames.COVARIATES});")
columns = [row[1] for row in cursor.fetchall()]
columns = [col for col in columns if col != primary_key]

query_sql = f"""
SELECT c.{primary_key}, {', '.join(f'c.{col}' for col in columns)} 
FROM {TableNames.COVARIATES} c INNER JOIN {TableNames.STATUS} s ON c.eid = s.eid
{exclusion_criteria};
"""

cursor.execute(query_sql)
columns = [description[0] for description in cursor.description]
data = cursor.fetchall()

df = pd.DataFrame(data, columns=columns)
df.to_csv("eligible_data.csv", index=False)

In [68]:
df_na = df.dropna()
df_na.to_csv("eligible_data_complete_case.csv", index=False)

In [71]:
# check the shape of all three csv files.
csv_full = pd.read_csv("full_data.csv")
csv_eligible = pd.read_csv("eligible_data.csv")
csv_eligible_complete = pd.read_csv("eligible_data_complete_case.csv")

print(csv_full.shape, csv_eligible.shape, csv_eligible_complete.shape)

(77888, 15) (35159, 15) (25594, 15)


# 